In [ ]:
target = pd.read_csv('target.csv')
data = pd.read_csv('data.csv')
test = pd.read_csv('test.csv')
vdata = pd.read_csv('vdata.csv')
vtarget = pd.read_csv('vtarget.csv')
wdata = pd.read_csv('whole_data.csv', header=None)
wtarget = pd.read_csv('whole_target.csv', header=None)

winedata = pd.read_csv('winedata.csv', header=None)

ccdata = pd.read_csv('creditcard.csv', header=None)

irisdata = pd.read_csv('irisdata.csv', header=None)
iristarget = pd.read_csv('iristarget.csv', header=None)

print('Data is ready !!')

In [ ]:
decode_dic = {0: 'fine_concrete',
              1: 'concrete',
              2: 'soft_tiles',
              3: 'tiled',
              4: 'soft_pvc',
              5: 'hard_tiles_large_space',
              6: 'carpet',
              7: 'hard_tiles',
              8: 'wood'}

In [ ]:
class Attention(Layer):
    def __init__(self, step_dim, W_regularizer=None, b_regularizer=None, W_constraint=None, b_constraint=None, bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)
        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)
        
    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = self.add_weight((input_shape[-1],), initializer=self.init, name='{}_W'.format(self.name), regularizer=self.W_regularizer, constraint=self.W_constraint)
        self.features_dim = input_shape[-1]
        if self.bias:
            self.b = self.add_weight((input_shape[1],), initializer='zero', name='{}_b'.format(self.name), regularizer=self.b_regularizer, constraint=self.b_constraint)
        else:
            self.b = None
        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim
        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))
        if self.bias: eij += self.b
        eij = K.tanh(eij)
        a = K.exp(eij)
        if mask is not None: a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [ ]:
def make_model():
    inp = Input(shape=(128, 10))
    x = Bidirectional(CuDNNLSTM(32, return_sequences=True))(inp)
    x = Attention(128)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(.5)(x)
    x = Dense(9, activation="softmax")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
def k_folds(X, y, X_test, k=5):
    folds = list(StratifiedKFold(n_splits=k).split(X, y))
    y_test = np.zeros((X_test.shape[0], 9))
    y_oof = np.zeros((X.shape[0]))
    
    for i, (train_idx, val_idx) in  enumerate(folds):
        print(f'Fold {i+1}')
        model = make_model()
        model.fit(X[train_idx], y[train_idx], batch_size=128, epochs=100, 
                  validation_data=[validation_part_df, y[val_idx]], verbose=0)
        
        pred_val = np.argmax(model.predict(X[val_idx]), axis=1)
        score = accuracy_score(pred_val, y[val_idx])
        y_oof[val_idx] = pred_val
        
        print(f'Scored {score:.3f} on validation data')
        
        y_test += model.predict(X_test)
        
    return y_oof, y_test  

In [ ]:
y_oof, y_test = k_folds(train_part_df, y_train, X_test, k=5)

In [ ]:
print(f'Local CV is {accuracy_score(y_oof, y_train): .4f}')

In [ ]:
y_test = np.argmax(y_test, axis=1)

In [ ]:
sub['surface'] = y_test
sub['surface'] = sub['surface'].map(decode_dic)
sub

In [ ]:
sub.to_csv('submission4.csv', index=False)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

train_x = pd.read_csv('X_train.csv')
train_y = pd.read_csv('y_train.csv')

num_test=500

def prepare_data(t):
    def f(d):
        d=d.sort_values(by=['measurement_number'])
        return pd.DataFrame({
         'lx':[ d['linear_acceleration_X'].values ],
         'ly':[ d['linear_acceleration_Y'].values ],
         'lz':[ d['linear_acceleration_Z'].values ],
         'ax':[ d['angular_velocity_X'].values ],
         'ay':[ d['angular_velocity_Y'].values ],
         'az':[ d['angular_velocity_Z'].values ],
         'ox':[ d['orientation_X'].values ],
         'oy':[ d['orientation_Y'].values ],
         'oz':[ d['orientation_Z'].values ],
         'ow':[ d['orientation_W'].values ],
        })

    t= t.groupby('series_id').apply(f)
    return t


def split_shuffle_groups(t):
    t= t.copy()

    # select randomly some groups (should be weighted by # of samples)

    aggcol='surface' # arbitrary; just to get size
    gstat= t.groupby('group_id')[aggcol].agg(np.size)
    gstat= gstat.reset_index()

    import random
    random.shuffle

    groups = list(zip(gstat['group_id'].values, gstat[aggcol].values))
    random.shuffle(groups)
    
    test_groups= set()
    c=0
    for gid,len in groups:
        if c>=num_test: break
        c+=len
        test_groups.add(gid)
    print("test groups:", test_groups)

    ctest = [ i for i,gid in enumerate(t['group_id']) if (gid in test_groups) ]
    ctrain = [ i for i,gid in enumerate(t['group_id']) if not (gid in test_groups) ]

    random.shuffle(ctrain)
    random.shuffle(ctest)

    return t.iloc[ctrain], t.iloc[ctest]


train= prepare_data(train_x)

# merge
train=pd.merge(train,train_y[['series_id','group_id','surface']],on='series_id')

train_part_df, validation_part_df= split_shuffle_groups(train)

print("training part of training data set:", train_part_df.describe())
print("validation part of training data set:",validation_part_df.describe())